In [1]:
import numpy as np
import pandas as pd

import time
from tqdm import tqdm

import glob
import re

In [29]:
file_list = glob.glob('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/*.csv')

In [30]:
file_list

['/Users/hwanghyeonjun/Documents/GitHub/data/selenium/터키 BIST 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/영국 3년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/스위스 2개월 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/프랑스 50년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/구리 선물 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/Dow Jones Shanghai 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/스위스 SMI 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/한국 30년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/중국 1년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/독일 5년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/일본 3년 채권 수익율 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/BTC_KRW Bithumb 내역.csv',
 '/Users/hwanghyeonjun/Documents/GitHub/data/selenium/미국 3개월 채권 수익율 내역.csv',
 '/Users/hwang

In [31]:
file_names = []
for path in tqdm(file_list):
	file_name = path.split('/')[-1]
	file_names.append(file_name)

100%|██████████| 220/220 [00:00<00:00, 455452.56it/s]


In [32]:
len(file_list), len(file_names)

(220, 220)

In [308]:
# # 데이터 형식 바꾸기
# def data_format_change(file_list: list, file_names: list):
# 	for idx, file in enumerate(tqdm(file_list)):
# 		date_regx = '\D?\s?'
# 		persent_regx = '\d+[%]$'
#
# 		prices = ['종가', '오픈', '고가', '저가']
#
# 		volumes = []
#
# 		load_df = pd.read_csv(file)
# 		df = load_df.astype(str)
#
# 		df.rename(columns={'변동 %': '변동'}, inplace=True)
#
# 		df['날짜'] = df['날짜'].apply(lambda x: re.sub(date_regx, '', x))
#
# 		for price in prices:
# 			df[price] = df[price].apply(lambda x: x.replace(',', '')).astype('float')
#
# 		df['거래량'] = df['거래량'].apply(lambda x: x.replace(',', ''))
#
# 		for volume in df['거래량'].values:
# 			cash_unit = list(volume)[-1]
#
# 			if str(volume) == 'nan':
# 				n = str(volume).replace('nan', '0')
# 				volumes.append(n)
# 			elif cash_unit == 'K':
# 				k = volume.replace(cash_unit, '')
# 				k_result = float(k) * 1000
# 				volumes.append(k_result)
# 			elif cash_unit == 'M':
# 				m = volume.replace(cash_unit, '')
# 				m_result = round(float(m) * 1000000)
# 				volumes.append(m_result)
# 			elif cash_unit == 'B':
# 				b = volume.replace(cash_unit, '')
# 				b_result = round(float(b) * 1000000000)
# 				volumes.append(b_result)
#
# 		df['거래량'] = volumes
#
# 		if not bool(re.match(persent_regx, list(df['변동'])[0])):
# 			df['변동'] = df['변동'].apply(lambda x: x.replace('%', '')).astype('float')
#
# 		df.sort_values(['날짜'], inplace=True)

In [309]:
data_format_change(file_list)

100%|██████████| 43/43 [00:00<00:00, 124.78it/s]


In [33]:

df_list = []
b = []

for idx, file in enumerate(file_list):
	date_regx = '\D?\s?'
	persent_regx = '\d+[%]$'

	prices = ['종가', '오픈', '고가', '저가']

	volumes = []

	load_df = pd.read_csv(file)
	df = load_df.astype(str)

	if str(list(load_df['날짜'].values)[0]) == '결과를 찾을 수 없습니다':
		print('결과를 찾을 수 없습니다')
		print(file_names[idx])
		print('='*100)
		file_list.pop(idx)
		file_names.pop(idx)
	else:
		pass

	df.rename(columns={'변동 %': '변동'}, inplace=True)

	df['날짜'] = df['날짜'].apply(lambda x: re.sub(date_regx, '', x))

	for price in prices:
		df[price] = df[price].apply(lambda x: x.replace(',', '')).astype('float')

	if '거래량' in list(df.columns):
		df['거래량'] = df['거래량'].apply(lambda x: x.replace(',', ''))

		for volume in df['거래량'].values:
			cash_unit = list(volume)[-1]

			if str(volume) == 'nan':
				n = str(volume).replace('nan', '0')
				volumes.append(n)
			elif cash_unit == 'K':
				k = volume.replace(cash_unit, '')
				k_result = round(float(k) * 1000)
				volumes.append(k_result)
			elif cash_unit == 'M':
				m = volume.replace(cash_unit, '')
				m_result = round(float(m) * 1000000)
				volumes.append(m_result)
			elif cash_unit == 'B':
				b = volume.replace(cash_unit, '')
				b_result = round(float(b) * 1000000000)
				volumes.append(b_result)

		df['거래량'] = volumes
	else:
		df.insert(5, '거래량', '0', allow_duplicates=False)

	df['변동'] = df['변동'].apply(lambda x: x.replace(',', ''))

	if not bool(re.match(persent_regx, list(df['변동'])[0])):
		df['변동'] = df['변동'].apply(lambda x: x.replace('%', '')).astype('float')

	df.sort_values(['날짜'], inplace=True)

	col_name = df.columns[1:]

	for col in col_name:
		df.rename(columns={col: file_names[idx] + '_' + col}, inplace=True)

	df_list.append(df)


결과를 찾을 수 없습니다
러시아 6개월 채권 수익율 내역.csv
결과를 찾을 수 없습니다
러시아 1주 채권 수익율 내역.csv
결과를 찾을 수 없습니다
러시아 1개월 채권 수익율 내역.csv
결과를 찾을 수 없습니다
터키 1년 채권 수익율 내역.csv
결과를 찾을 수 없습니다
러시아 3개월 채권 수익율 내역.csv
결과를 찾을 수 없습니다
러시아 오버나잇 채권 수익율 내역.csv
결과를 찾을 수 없습니다
러시아 2주 채권 수익율 내역.csv
결과를 찾을 수 없습니다
러시아 2개월 채권 수익율 내역.csv


In [34]:
len(file_list), len(file_names)

(212, 212)

In [35]:
file_names

['터키 BIST 내역.csv',
 '영국 3년 채권 수익율 내역.csv',
 '스위스 2개월 채권 수익율 내역.csv',
 '프랑스 50년 채권 수익율 내역.csv',
 '구리 선물 내역.csv',
 'Dow Jones Shanghai 내역.csv',
 '스위스 SMI 내역.csv',
 '한국 30년 채권 수익율 내역.csv',
 '중국 1년 채권 수익율 내역.csv',
 '독일 5년 채권 수익율 내역.csv',
 '일본 3년 채권 수익율 내역.csv',
 'BTC_KRW Bithumb 내역.csv',
 '미국 3개월 채권 수익율 내역.csv',
 '영국 6년 채권 수익율 내역.csv',
 '닛케이 내역.csv',
 '일본 6년 채권 수익율 내역.csv',
 'IDR_KRW 내역.csv',
 'CNY_KRW 내역.csv',
 '호주 S&P_ASX 내역.csv',
 '중국 30년 채권 수익율 내역.csv',
 '한국 2년 채권 수익율 내역.csv',
 '영국 3개월 채권 수익율 내역.csv',
 '일본 9년 채권 수익율 내역.csv',
 'MXN_KRW 내역.csv',
 '스위스 4년 채권 수익율 내역.csv',
 'DAX 내역.csv',
 '사우디아라비아 Tadawul 내역.csv',
 'RTSI 지수 내역.csv',
 '다우존스 내역.csv',
 '영국 12년 채권 수익율 내역.csv',
 '영국 9년 채권 수익율 내역.csv',
 'PHP_KRW 내역.csv',
 'AED_KRW 내역.csv',
 '영국 FTSE 내역.csv',
 'Nifty 50 내역.csv',
 '스위스 1년 채권 수익율 내역.csv',
 '터키 5년 채권 수익율 내역.csv',
 'S&P 500 내역.csv',
 '러시아 5년 채권 수익율 내역.csv',
 '프랑스 5년 채권 수익율 내역.csv',
 '미국 5년 채권 수익율 내역.csv',
 'EUR_KRW 내역.csv',
 '일본 40년 채권 수익율 내역.csv',
 '영국 50년 채권 수익율 내역.csv',
 '프랑스 3개월 채

In [10]:
# # 날짜 순서로 오름차순정렬
#
# def dataframe_srot(file_list):
# 	file_names = []
#
# 	for path in tqdm(file_list):
# 		file_name = path.split('/')[-1]
# 		file_names.append(file_name)
#
# 	for idx, file in tqdm(enumerate(file_list)):
# 		df = pd.read_csv(file)
# 		df.sort_values(['날짜'], inplace=True)
# 		df['날짜'] = df['날짜'].apply(lambda x: x.replace(' ', ''))
# 		df.to_csv('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/' + file_names[idx], index=False)

In [47]:
# dataframe_srot(rate_file_list)

100%|██████████| 23/23 [00:00<00:00, 450789.68it/s]
23it [00:00, 270.72it/s]


In [3]:
data = pd.read_csv('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/러시아 6개월 채권 수익율 내역.csv')

In [4]:
data

,날짜,종가,오픈,고가,저가,변동 %
0,결과를 찾을 수 없습니다,NaN,NaN,NaN,NaN,NaN


In [13]:
data.insert(5, '거래량', '0', allow_duplicates=False)

In [25]:
data.drop('거래량', axis=1, inplace=True)

In [26]:
if '거래량' in list(data.columns):
	print('있음')
else:
	print('없음')

없음


In [7]:
last_df = data.iloc[-1:, :]

In [8]:
last_df

,날짜,종가,오픈,고가,저가,변동 %
745,2022-11-11,2.286,2.24,2.328,2.228,2.24%


In [111]:
str(list(data['날짜'].values)[0])

'결과를 찾을 수 없습니다'

In [120]:
test_data = pd.read_csv('/Users/hwanghyeonjun/Documents/GitHub/data/selenium/러시아 오버나잇 채권 수익율 내역.csv')

In [121]:
test_data

,날짜,종가,오픈,고가,저가,변동 %
0,결과를 찾을 수 없습니다,NaN,NaN,NaN,NaN,NaN


In [122]:
if not str(list(test_data['날짜'].values)[0]) == '결과를 찾을 수 없습니다':
	print('결과 있음')
else:
	print('결과 없음')

결과 없음
